# Search-Based Software Engineering Exercise
## Exercise 02 - CSP
<center>
    
    Stefan Jahns
    
    Softwaresysteme - Summer Term 2024
    
</center>
<center>
    <img src='uni-leipzig.png' style="height:5em" >   <img src='SOSY-Logo.png' style="height:5em">
    
</center>

## Scope of this excercise
 - apply CSP solvers to configurable software systems
 - mini-lesson on Feature Models

For many use cases, it is useful to represent configurations as binary vectors s.a. 
[1 0 1 0 1 1 1]

<center>
    <img src='VP9.png' style="height:20em" />  
</center>

## Short introduction to pycosat
```
pip install pycosat
```

In [1]:
import pycosat

### Supplying constraints

```pycosat.solve(constraints)``` takes a list of CNF clauses and yields a valid solution

Example with 3 features

In [3]:
constraints = []
constraints.append([1])
constraints.append([2,-3])
constraints

[[1], [2, -3]]

In [4]:
pycosat.solve(constraints)

[1, 2, -3]

### Checking for Satisfyability

In [5]:
solution = pycosat.solve(constraints)
if solution == "UNSAT":
    print("No solutions found!")
else:
    print("Solution:", solution)

Solution: [1, 2, -3]


### Adding constraints

In [6]:
constraints

[[1], [2, -3]]

In [7]:
constraints.append([-2])
pycosat.solve(constraints)

[1, -2, -3]

In [8]:
constraints.append([2])
pycosat.solve(constraints)

'UNSAT'

### Constraints in the wild

# Material

In [9]:
import pycosat
import numpy as np
import copy

In [10]:
def parse_dimacs(path):
    """
    A function to parse a provided DIMACS-file.

    Args:
        path (str): The DIMACS-file's file path

    Returns:
        A list of lists containing all of the DIMACS-file's constrains. Each constraint is represented by a seperate sub-list.
    """
    dimacs = list()
    dimacs.append(list())
    with open(path) as mfile:
        for line in mfile:
            tokens = line.split()
            if len(tokens) != 0 and tokens[0] not in ("p", "c"):
                for tok in tokens:
                    lit = int(tok)
                    if lit == 0:
                        dimacs.append(list())
                    else:
                        dimacs[-1].append(lit)
    assert len(dimacs[-1]) == 0
    dimacs.pop()
    return dimacs

In [11]:
def transform2binary(sol):
    """
    A function which takes a valid variant, consisting of positive and negative integers and transforming it into binary values
    Args:
        sol (list): A list that contains one valid variant, represented by positve and negative integers

    Returns:
        A list that contains the valid variants transformed into binary, where negative integers are now represented as 0 and positive integers as 1
    """
    sol = sorted(sol, key=abs)
    for index, elem in enumerate(sol):
        if float(elem) < 0:
            sol[index] = 0
        else:
            sol[index] = 1
    return sol

In [12]:
def get_valid_variants(constraint_list, size):
    """
    A function to compute the valid variants of a model.

    Args:
        constraint_list (list): All constrains provided for the model.
        size (int): The desired number of variants for the model.

    Returns:
        A list of variants, which satisfy the provided constrains. Each entry represents one variant.
    """
    my_constraints = copy.deepcopy(constraint_list)
    variants = []
    for i in range(0, size):
            solution = pycosat.solve(my_constraints)
            if solution == "UNSAT":
                print("No more solutions found!")
                break
            binary_solution = transform2binary(solution)
            print("Pycosat Literals:", solution, "Binary Vector Solution", binary_solution)
            variants.append(binary_solution)
            #TODO: solve exercise
            
    return variants

In [19]:
def score(a,b):
    """
    returns root-mean-squared distance between two populatons (high is dissimilar, 0.0 is identical)
    """
    def get_ft_frequencies(pop):
        pop_np = np.array(pop)
        pop_np = pop_np / np.sum(pop_np)
        return pop_np.sum(axis=0)
    
    a_freqs = get_ft_frequencies(a)
    b_freqs = get_ft_frequencies(b)
    statistic = np.sqrt(np.sum(np.power(a_freqs - b_freqs, 2)))
    return statistic

## Example Usage

In [13]:
clauses = parse_dimacs("VP9.dimacs") 

In [14]:
variants = get_valid_variants(clauses, 2)
variants

Pycosat Literals: [1, 2, 3, -4, 5, -6, -7, -8, 9, -10, -11, -12, 13, -14, -15, -16, 17, 18, 19, -20, -21, 22, 23, -24, -25] Binary Vector Solution [1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0]
Pycosat Literals: [1, 2, 3, -4, 5, -6, -7, -8, 9, -10, -11, -12, 13, -14, -15, -16, 17, 18, 19, -20, -21, 22, 23, -24, -25] Binary Vector Solution [1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0]


[[1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0],
 [1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0]]

In [15]:
import pickle
with open( "dump.p", "wb" ) as file:
    pickle.dump( variants,  file)

In [16]:
with open( "dump.p", "rb" ) as file:
    old_data = pickle.load(file)
old_data

[[1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0],
 [1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0]]

In [21]:
pop_A = (
    [1,0,0],
    [1,1,0]
)
pop_B = (
    [0,0,0],
    [0,1,0]
)
pop_C = (
    [0,0,0],
    [0,1,1]
)

In [23]:
score(pop_B, pop_C)

0.7071067811865476